# Capstone Project: Data Collection     

29/11/2020

We collect restaurant data via the Foursquare API.

In [1]:
import pandas as pd
import numpy as np
import requests 
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!pip install folium
import folium # map rendering library


In [17]:
# Coordinates of Toronto
latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
#for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
#    label = '{}, {}'.format(neighborhood, borough)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_dublin)  
    
map_toronto

In [3]:
# @hidden_cell
CLIENT_ID = '34CRGBIL11CRLO5BNFSRKHWNRQT1GWTB4QKGYG5GLQYM3JKQ' # your Foursquare ID
CLIENT_SECRET = 'HRPY3HMJC1FSPYTC2VRVUANCY1JY20NBRA5400RQYOEOFCNB' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
VERSION = '20201123' # Foursquare API version
RADIUS=500
LIMIT = 50 # A default Foursquare API limit value

In [4]:
# List of categories
afghan = '503288ae91d4c4b30a586d67'
african = '4bf58dd8d48988d1c8941735'
american = '4bf58dd8d48988d14e941735'
asian = '4bf58dd8d48988d142941735'
australian = '4bf58dd8d48988d169941735'
austrian = '52e81612bcbc57f1066b7a01'
bangladesh = '5e179ee74ae8e90006e9a746'
belgium = '52e81612bcbc57f1066b7a02'
cajun = '4bf58dd8d48988d17a941735'
caribbean = '4bf58dd8d48988d144941735'
caucasian = '5293a7d53cf9994f4e043a45'
czech = '52f2ae52bcbc57f1066b8b81'
dutch = '5744ccdfe4b0c0459246b4d0'
eastEurope = '4bf58dd8d48988d109941735'
english = '52e81612bcbc57f1066b7a05'
french = '4bf58dd8d48988d10c941735'
german = '4bf58dd8d48988d10d941735'
greek = '4bf58dd8d48988d10e941735'
hungary = '52e81612bcbc57f1066b79fa'
indian = '4bf58dd8d48988d10f941735'
italian = '4bf58dd8d48988d110941735'
jewish = '52e81612bcbc57f1066b79fd'
latin = '4bf58dd8d48988d1be941735'
meditarranean = '4bf58dd8d48988d1c0941735'
mexican = '4bf58dd8d48988d1c1941735'
middleEast = '4bf58dd8d48988d115941735'
modernEuropean = '52e81612bcbc57f1066b79f9'
pakistan = '52e81612bcbc57f1066b79f8'
polish = '52e81612bcbc57f1066b7a04'
portugal = '4def73e84765ae376e57713a'
russia = '5293a7563cf9994f4e043a44'
scandinavia = '4bf58dd8d48988d1c6941735'
scotland = '5744ccdde4b0c0459246b4a3'
slovak = '56aa371be4b08b9a8d57355a'
spain = '4bf58dd8d48988d150941735'
srilanka = '5413605de4b0ae91d18581a9'
swiss = '4bf58dd8d48988d158941735'
turkey = '4f04af1f2fb6e1c99f3db0bb'
ukraine = '52e928d0bcbc57f1066b7e96'

category_list = [afghan,african,american,asian,australian,austrian,bangladesh,belgium,cajun,caribbean,caucasian,czech,dutch,eastEurope,english,
                 french,german,greek,hungary,indian,italian,jewish,latin,meditarranean,mexican,middleEast,modernEuropean,pakistan,polish,
                 portugal,russia,scandinavia,scotland,slovak,spain,srilanka,swiss,turkey,ukraine] 

CATEGORIES = ','.join(category_list)


In [5]:
def getVenues(latitude, longitude, RADIUS=500):
    
    url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse'.format(
            CATEGORIES,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS,
            LIMIT)
    
    response = requests.get(url)
    results = response.json()
    
    if response.status_code == 200:
        venues = results['response']['venues']
    else:
        venues = []
        print("HTTP-{}".format(response.status_code))
        
    venue_list = []
    
    number_of_venues = len(venues)
    
    for i in range(number_of_venues):
        
        try:
            venue_id = venues[i]['id']
            venue_name = venues[i]['name']
            lat = venues[i]['location']['lat']
            lng = venues[i]['location']['lng']
            category = venues[i]['categories'][0]['name']
            
            record = (latitude, longitude, number_of_venues, i, venue_id, venue_name, lat, lng, category)
        except:
            record = (latitude, longitude, number_of_venues, i, '', '', '', '', '')
        
        venue_list.append(record)
        
        
    return( venue_list )


# Toronto

In [42]:
# Coordinates of Toronto
latitude = 43.6532
longitude = -79.3832

# Box boundaries
y_max = latitude + 0.17
y_min = latitude - 0.05
x_max = longitude + 0.2
x_min = longitude - 0.2


lat_centre = (y_max+y_min)/2.0

y_step = 1.0/(110.574)
x_step = 1.0/(111.32*np.cos(2.0*np.pi*lat_centre/360.0))

num_x_points = int((x_max-x_min)/x_step)
num_y_points = int((y_max-y_min)/y_step)

lats = np.linspace(x_min,x_max,num_x_points)
longs = np.linspace(y_min,y_max,num_y_points)

' Create matrix of coordinate values '
coord_list = []
for x in longs:
    for y in lats:
        
        coord_list.append( (x,y) )

print("Number of points = {}".format(len(coord_list)))

Number of points = 768


In [43]:
# Coordinates of Toronto
latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Retrieve data from Foursquare

In [47]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('toronto_20201204_2105.csv',encoding='utf-8')

lat = 43.6032 long = -79.5832 Number of Venues = 6
lat = 43.6032 long = -79.5702967742 Number of Venues = 4
lat = 43.6032 long = -79.5573935484 Number of Venues = 1
lat = 43.6032 long = -79.5444903226 Number of Venues = 4
lat = 43.6032 long = -79.5315870968 Number of Venues = 3
lat = 43.6032 long = -79.518683871 Number of Venues = 1
lat = 43.6032 long = -79.5057806452 Number of Venues = 17
lat = 43.6032 long = -79.4928774194 Number of Venues = 1
lat = 43.6032 long = -79.4799741935 Number of Venues = 0
lat = 43.6032 long = -79.4670709677 Number of Venues = 0
lat = 43.6032 long = -79.4541677419 Number of Venues = 0
lat = 43.6032 long = -79.4412645161 Number of Venues = 0
lat = 43.6032 long = -79.4283612903 Number of Venues = 0
lat = 43.6032 long = -79.4154580645 Number of Venues = 1
lat = 43.6032 long = -79.4025548387 Number of Venues = 0
lat = 43.6032 long = -79.3896516129 Number of Venues = 0
lat = 43.6032 long = -79.3767483871 Number of Venues = 0
lat = 43.6032 long = -79.3638451613 N

lat = 43.6414608696 long = -79.5057806452 Number of Venues = 0
lat = 43.6414608696 long = -79.4928774194 Number of Venues = 0
lat = 43.6414608696 long = -79.4799741935 Number of Venues = 0
lat = 43.6414608696 long = -79.4670709677 Number of Venues = 0
lat = 43.6414608696 long = -79.4541677419 Number of Venues = 1
lat = 43.6414608696 long = -79.4412645161 Number of Venues = 43
lat = 43.6414608696 long = -79.4283612903 Number of Venues = 32
lat = 43.6414608696 long = -79.4154580645 Number of Venues = 38
lat = 43.6414608696 long = -79.4025548387 Number of Venues = 45
lat = 43.6414608696 long = -79.3896516129 Number of Venues = 50
lat = 43.6414608696 long = -79.3767483871 Number of Venues = 39
lat = 43.6414608696 long = -79.3638451613 Number of Venues = 2
lat = 43.6414608696 long = -79.3509419355 Number of Venues = 0
lat = 43.6414608696 long = -79.3380387097 Number of Venues = 0
lat = 43.6414608696 long = -79.3251354839 Number of Venues = 0
lat = 43.6414608696 long = -79.3122322581 Number 

lat = 43.6797217391 long = -79.4670709677 Number of Venues = 9
lat = 43.6797217391 long = -79.4541677419 Number of Venues = 6
lat = 43.6797217391 long = -79.4412645161 Number of Venues = 34
lat = 43.6797217391 long = -79.4283612903 Number of Venues = 42
lat = 43.6797217391 long = -79.4154580645 Number of Venues = 7
lat = 43.6797217391 long = -79.4025548387 Number of Venues = 11
lat = 43.6797217391 long = -79.3896516129 Number of Venues = 18
lat = 43.6797217391 long = -79.3767483871 Number of Venues = 1
lat = 43.6797217391 long = -79.3638451613 Number of Venues = 6
lat = 43.6797217391 long = -79.3509419355 Number of Venues = 50
lat = 43.6797217391 long = -79.3380387097 Number of Venues = 39
lat = 43.6797217391 long = -79.3251354839 Number of Venues = 21
lat = 43.6797217391 long = -79.3122322581 Number of Venues = 2
lat = 43.6797217391 long = -79.2993290323 Number of Venues = 7
lat = 43.6797217391 long = -79.2864258065 Number of Venues = 7
lat = 43.6797217391 long = -79.2735225806 Number

lat = 43.7179826087 long = -79.4283612903 Number of Venues = 9
lat = 43.7179826087 long = -79.4154580645 Number of Venues = 0
lat = 43.7179826087 long = -79.4025548387 Number of Venues = 7
lat = 43.7179826087 long = -79.3896516129 Number of Venues = 0
lat = 43.7179826087 long = -79.3767483871 Number of Venues = 0
lat = 43.7179826087 long = -79.3638451613 Number of Venues = 1
lat = 43.7179826087 long = -79.3509419355 Number of Venues = 0
lat = 43.7179826087 long = -79.3380387097 Number of Venues = 7
lat = 43.7179826087 long = -79.3251354839 Number of Venues = 0
lat = 43.7179826087 long = -79.3122322581 Number of Venues = 0
lat = 43.7179826087 long = -79.2993290323 Number of Venues = 5
lat = 43.7179826087 long = -79.2864258065 Number of Venues = 3
lat = 43.7179826087 long = -79.2735225806 Number of Venues = 0
lat = 43.7179826087 long = -79.2606193548 Number of Venues = 4
lat = 43.7179826087 long = -79.247716129 Number of Venues = 3
lat = 43.7179826087 long = -79.2348129032 Number of Venu

lat = 43.7562434783 long = -79.3896516129 Number of Venues = 0
lat = 43.7562434783 long = -79.3767483871 Number of Venues = 0
lat = 43.7562434783 long = -79.3638451613 Number of Venues = 5
lat = 43.7562434783 long = -79.3509419355 Number of Venues = 22
lat = 43.7562434783 long = -79.3380387097 Number of Venues = 0
lat = 43.7562434783 long = -79.3251354839 Number of Venues = 2
lat = 43.7562434783 long = -79.3122322581 Number of Venues = 16
lat = 43.7562434783 long = -79.2993290323 Number of Venues = 2
lat = 43.7562434783 long = -79.2864258065 Number of Venues = 3
lat = 43.7562434783 long = -79.2735225806 Number of Venues = 8
lat = 43.7562434783 long = -79.2606193548 Number of Venues = 6
lat = 43.7562434783 long = -79.247716129 Number of Venues = 1
lat = 43.7562434783 long = -79.2348129032 Number of Venues = 3
lat = 43.7562434783 long = -79.2219096774 Number of Venues = 2
lat = 43.7562434783 long = -79.2090064516 Number of Venues = 0
lat = 43.7562434783 long = -79.1961032258 Number of Ve

lat = 43.7945043478 long = -79.3509419355 Number of Venues = 9
lat = 43.7945043478 long = -79.3380387097 Number of Venues = 1
lat = 43.7945043478 long = -79.3251354839 Number of Venues = 5
lat = 43.7945043478 long = -79.3122322581 Number of Venues = 4
lat = 43.7945043478 long = -79.2993290323 Number of Venues = 1
lat = 43.7945043478 long = -79.2864258065 Number of Venues = 0
lat = 43.7945043478 long = -79.2735225806 Number of Venues = 13
lat = 43.7945043478 long = -79.2606193548 Number of Venues = 4
lat = 43.7945043478 long = -79.247716129 Number of Venues = 17
lat = 43.7945043478 long = -79.2348129032 Number of Venues = 12
lat = 43.7945043478 long = -79.2219096774 Number of Venues = 2
lat = 43.7945043478 long = -79.2090064516 Number of Venues = 1
lat = 43.7945043478 long = -79.1961032258 Number of Venues = 4
lat = 43.7945043478 long = -79.1832 Number of Venues = 0
lat = 43.8040695652 long = -79.5832 Number of Venues = 0
lat = 43.8040695652 long = -79.5702967742 Number of Venues = 0
la

In [48]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 16 sites


array([[ 43.64146087, -79.38965161],
       [ 43.65102609, -79.41545806],
       [ 43.65102609, -79.40255484],
       [ 43.65102609, -79.38965161],
       [ 43.65102609, -79.37674839],
       [ 43.6605913 , -79.41545806],
       [ 43.6605913 , -79.40255484],
       [ 43.6605913 , -79.37674839],
       [ 43.67015652, -79.38965161],
       [ 43.67972174, -79.35094194],
       [ 43.70841739, -79.40255484],
       [ 43.7658087 , -79.41545806],
       [ 43.77537391, -79.41545806],
       [ 43.78493913, -79.27352258],
       [ 43.80406957, -79.28642581],
       [ 43.8232    , -79.29932903]])

## Calculate new site locations at breached points

In [49]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()


In [50]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/4.0)
    xright = x + (x_step/4.0)
    ytop = y + (y_step/4.0)
    ybottom = y - (y_step/4.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [51]:
# Coordinates of Toronto
latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Retrieve info from Foursquare

In [52]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=250)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('toronto_20201204_2215.csv',encoding='utf-8')

lat = 43.6414608696 long = -79.3896516129 Number of Venues = 5
lat = 43.6437217989 long = -79.3927549193 Number of Venues = 17
lat = 43.6391999402 long = -79.3927549193 Number of Venues = 10
lat = 43.6437217989 long = -79.3865483065 Number of Venues = 18
lat = 43.6391999402 long = -79.3865483065 Number of Venues = 6
lat = 43.651026087 long = -79.4154580645 Number of Venues = 4
lat = 43.6532870163 long = -79.4185618651 Number of Venues = 18
lat = 43.6487651576 long = -79.4185618651 Number of Venues = 23
lat = 43.6532870163 long = -79.4123542639 Number of Venues = 13
lat = 43.6487651576 long = -79.4123542639 Number of Venues = 3
lat = 43.651026087 long = -79.4025548387 Number of Venues = 13
lat = 43.6532870163 long = -79.4056586393 Number of Venues = 28
lat = 43.6487651576 long = -79.4056586393 Number of Venues = 18
lat = 43.6532870163 long = -79.3994510381 Number of Venues = 50
lat = 43.6487651576 long = -79.3994510381 Number of Venues = 29
lat = 43.651026087 long = -79.3896516129 Numbe

In [53]:
print("Limit breached in {} sites".format(sum(df_venues['num_venues']==50)/50))
np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0)

Limit breached in 14 sites


array([[ 43.64876516, -79.38654781],
       [ 43.64876516, -79.37985219],
       [ 43.64876516, -79.37364459],
       [ 43.65102609, -79.37674839],
       [ 43.65328702, -79.39945104],
       [ 43.65328702, -79.39275541],
       [ 43.65833038, -79.37985268],
       [ 43.66789559, -79.38654682],
       [ 43.67015652, -79.38965161],
       [ 43.77537391, -79.41545806],
       [ 43.77763484, -79.41234782],
       [ 43.80406957, -79.28642581],
       [ 43.80633049, -79.28953755],
       [ 43.8232    , -79.29932903]])

## Calculate new site locations at breached points

In [54]:
zoom_coord_list = np.unique(df_venues[df_venues['num_venues']==50][['lat_centre','long_centre']], axis=0).tolist()



In [55]:
coord_list = []

for coord in zoom_coord_list:
    
    x = coord[1]
    y = coord[0]
    
    y_step = 1.0/(110.574)
    x_step = 1.0/(111.32*np.cos(2.0*np.pi*y/360.0))

    xleft = x - (x_step/8.0)
    xright = x + (x_step/8.0)
    ytop = y + (y_step/8.0)
    ybottom = y - (y_step/8.0)
    
    coord_list.append( (y,x) )
    coord_list.append( (ytop,xleft) )
    coord_list.append( (ybottom,xleft) )
    coord_list.append( (ytop,xright) )
    coord_list.append( (ybottom,xright) )

In [56]:
# Coordinates of Toronto
latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for coord in coord_list:
    lat = coord[0]
    lng = coord[1]
    label = '{} {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Retrieve info from Foursquare

In [7]:
record_list = []

for coord in coord_list:
    
    lat = coord[0]
    lng = coord[1]
    
    venue_details = getVenues(lat, lng, RADIUS=100)
    
    record_list.extend(venue_details)
    
    print("lat = {} long = {} Number of Venues = {}".format(lat,lng,len(venue_details)))

    
col_names = ['lat_centre','long_centre','num_venues','venue_iter','venue_id','venue_name','lat_venue','long_venue','category']
df_venues = pd.DataFrame.from_records(record_list,columns=col_names)  

df_venues.to_csv('toronto_20201205_0953.csv',encoding='utf-8')

lat = 43.6487651576 long = -79.3865478123 Number of Venues = 10
lat = 43.6498956223 long = -79.3880996542 Number of Venues = 17
lat = 43.647634693 long = -79.3880996542 Number of Venues = 11
lat = 43.6498956223 long = -79.3849959704 Number of Venues = 6
lat = 43.647634693 long = -79.3849959704 Number of Venues = 13
lat = 43.6487651576 long = -79.3798521877 Number of Venues = 10
lat = 43.6498956223 long = -79.3814040296 Number of Venues = 11
lat = 43.647634693 long = -79.3814040296 Number of Venues = 4
lat = 43.6498956223 long = -79.3783003458 Number of Venues = 12
lat = 43.647634693 long = -79.3783003458 Number of Venues = 19
lat = 43.6487651576 long = -79.3736445865 Number of Venues = 8
lat = 43.6498956223 long = -79.3751964284 Number of Venues = 12
lat = 43.647634693 long = -79.3751964284 Number of Venues = 10
lat = 43.6498956223 long = -79.3720927446 Number of Venues = 4
lat = 43.647634693 long = -79.3720927446 Number of Venues = 3
lat = 43.651026087 long = -79.3767483871 Number of 

## Combine all files into single file

In [8]:
import os
files = os.listdir('.')

file_pattern = 'toronto_'

venue_files = []

' Obtain all files holding venue info '
for filename in files:
    if file_pattern in filename:
        venue_files.append(filename)

' Read in files into dataframe '
df_venues = pd.DataFrame()
for filename in venue_files:
    df = pd.read_csv(filename,encoding='utf-8')
    print( "File: {} Records: {}".format(filename,len(df)) )
    df_venues = df_venues.append(df, ignore_index = True) 

File: toronto_20201205_0953.csv Records: 696
File: toronto_20201204_2215.csv Records: 1805
File: toronto_20201204_2105.csv Records: 4109


## Remove duplicate entries and clean up data

In [10]:
df_venues.sort_values("venue_id", inplace = True)
df_venues.drop_duplicates(subset ="venue_id", keep = False, inplace = True)
df_venues.to_csv('toronto_all.csv',encoding='utf-8')

In [12]:
' Keep only venues with "Restaurant" in their category title '
df_venues['restaurant'] = df_venues['category'].apply(lambda x: True if 'Restaurant' in x else False)
df_venues = df_venues[df_venues['restaurant'] == True]
df_venues.drop('restaurant', axis=1, inplace=True)
df_venues.to_csv('toronto_restaurants.csv',encoding='utf-8')

In [33]:
from folium.plugins import FastMarkerCluster

# Coordinates of Toronto
latitude = 43.6532
longitude = -79.3832

# create map of Zurich using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

FastMarkerCluster(data=list(zip(df_venues['lat_venue'],df_venues['long_venue']))).add_to(map_toronto)

folium.LayerControl().add_to(map_toronto);

map_toronto